I refer to this notebook for making this kernel. Thank you @shogosuzuki !! https://www.kaggle.com/shogosuzuki/optuna-lightgbm-onehotencoder-lr-0-001

# **한국인을 위한 Tabular Playground Feb. 베이스라인**

이 competition은 Kaggle에서 매 달마다 진행하는 Tabular Playground 시리즈의 2월 대회입니다. Titanic이나 mnist 등 입문격의 대회보다는 어렵지만, 상금이 걸린 대회보다는 훨씬 쉬운, 머신러닝 초보자들이 참여하기 좋은 대회입니다. 

이 대회는 10개의 문자로 이루어진 레이블과 14개의 숫자 레이블을 통하여 하나의 Target을 예측하는 문제입니다. 간단한 DNN부터 Random Forest와 같은 Tree-based 모델, XGBoost와 같은 Gradient Boosting 모델 등을 이용할 수 있습니다. 이 베이스라인 코드에서는 LightGBM 모델을 사용하겠습니다. LGBM 모델에 관한 자세한 내용은 이 링크를 참고해주시기 바랍니다. https://nurilee.com/2020/04/03/lightgbm-definition-parameter-tuning/

입문자 분들을 위하여 라이브러리 import를 제외한 코드 하나하나 주석으로 설명을 첨부하였습니다. 

질문은 언제든지 댓글에 남겨주세요! numpy, pandas, scikit-learn, LGBM은 공식 문서를 참고하면 더욱 정확한 정보를 빠르게 찾을 수 있습니다.

* numpy - https://numpy.org/doc/1.19/
* pandas - https://pandas.pydata.org/pandas-docs/stable/reference/index.html
* scikit-learn - https://scikit-learn.org/stable/modules/classes.html
* LGBM - https://lightgbm.readthedocs.io/en/latest/

# **라이브러리 import**

In [ ]:
import numpy as np 
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import category_encoders as ce

# **데이터 전처리**

In [ ]:
df_train = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/train.csv") #train 데이터를 로드합니다
df_test = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/test.csv") #test 데이터를 로드합니다
df_sample = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv") #submission 파일 예시를 로드합니다

In [ ]:
df_train.drop("id", axis=1, inplace=True) #train 데이터에서 id 열을 버립니다. inplace가 True면 복사본을 반환하지 않습니다.
df_test.drop("id", axis=1, inplace=True) #test 데이터에서 id 열을 버립니다.

In [ ]:
cat_labels = [f"cat{i}" for i in range(10)] #cat0부터 cat9까지 10개의 문자로 이루어진 레이블을 정의합니다. 

데이터 전처리 과정에서 OneHotEncoder를 사용합니다. 문자로 이루어진 레이블은 바로 계산할 수 없기 때문에 숫자로 바꾸어주어야 합니다.
OneHotEncoder는 각 레이블 구성 요소의 종류만큼 새로운 레이블을 만들고, 각 종류에 해당하는지를 0과 1로 표현하는 방식으로 문자를 숫자로 변환합니다. 
가장 정확도가 높지만 데이터의 양이 많아질 수 있다는 단점도 있습니다. 
예를 들어, A와 B로 이루어진 레이블을 OneHotEncoder를 사용해서 변환하면 A는 [1,0], B는 [0,1]과 같은 형식으로 표현하게 됩니다. 

In [ ]:
onehot_encoder = ce.one_hot.OneHotEncoder() #OneHotEncoder를 정의합니다.
onehot_encoder.fit(pd.concat([df_train[cat_labels], df_test[cat_labels]], axis=0)) #train 데이터와 test 데이터를 합친 후 OneHotEncoding을 실행합니다.
train_ohe = onehot_encoder.transform(df_train[cat_labels]) #train 데이터에 OneHotEncoding을 적용한 train_ohe를 만듭니다.
test_ohe = onehot_encoder.transform(df_test[cat_labels]) #test 데이터에 OneHotEncoding을 적용한 test_ohe를 만듭니다.
train_ohe.columns = [f"OHE_{col}" for col in train_ohe] #OneHotEncoder는 각 레이블의 이름을 정해주지 않기 때문에, 레이블의 이름을 정해줍니다. 
test_ohe.columns = [f"OHE_{col}" for col in test_ohe] #레이블의 이름을 정해줍니다. 

In [ ]:
numerical_labels = [f"cont{i}" for i in range(14)] #cont0부터 cont13까지 14개의 숫자로 이루어진 레이블을 정의합니다. 

In [ ]:
train_x = pd.concat([df_train[numerical_labels],train_ohe], axis=1) #OneHotEncoding을 한 문자 label과 숫자 label을 합쳐 최종 train 데이터를 만듭니다.

In [ ]:
test_x = pd.concat([df_test[numerical_labels],test_ohe], axis=1) #최종 train 데이터를 만듭니다.

In [ ]:
train_y = df_train["target"] #train 데이터의 Target 값을 train_y로 정의해줍니다. 

# **모델 훈련 (교차 검증 포함)**

교차 검증 (Cross Validation)이란 train 데이터에서 실제로 학습시키는 데이터와 확인을 위한 validation 데이터를 여러번 분리하여 학습합니다. 이를 통하여 모델의 성능 평가를 일반화하고, 오버피팅을 방지할 수도 있습니다. 여기서는 KFold를 5개의 fold로 나누는데, 이렇게 되면 20%의 train 데이터를 validation에 사용하며, 5번 모두 다른 20%의 데이터로 validation을 합니다. 자세한 내용은 이 링크를 참고하시기 바랍니다. https://3months.tistory.com/321

In [ ]:
folds = KFold(n_splits=5, shuffle=True, random_state=2021) #5 fold의 KFold를 정의합니다. random_state는 임의대로 정의하여도 됩니다. 

In [ ]:
lgb_params = dict({'objective': 'regression', #LGBM 모델의 다양한 parameter들을 정의합니다. 이 문제는 회귀 문제이므로 regression으로 정의합니다. 
 'metric': 'rmse', #이 대회에서는 평가에 RMSE를 사용합니다. 
 'verbosity': -1, #모델 훈련 상태를 얼마만큼 알려줄 것인지 설정합니다. 
 'learning_rate': 0.01, #learning rate를 설정합니다
 'early_stopping_round' : 200, #200번의 훈련 동안 모델의 성능에 향상이 없으면 학습을 중지합니다. 
 'num_iterations' : 20000}) #최대 20000번 훈련합니다. 

In [ ]:
oof_preds = np.zeros_like(train_y) #validation의 예측값을 저장할 DataFrame을 만듭니다.
models = [] #5개의 모델을 저장할 list를 만듭니다.
for tr_idx, va_idx in folds.split(train_x): #5번 상호 검증을 수행하며 훈련할 for 반복문을 만듭니다. 
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx] #예측에 사용할 x 값들에서 validation 데이터를 분리합니다.
    tr_y, va_y = train_y[tr_idx], train_y[va_idx] #예측할 데이터인 y 값들에서 validation 데이터를 분리합니다.
            
    lgb_train_dataset = lgb.Dataset(tr_x, tr_y) #train 데이터를 LGBM용 데이터셋으로 변환합니다. 자세한 내용은 공식 문서를 참조하세요. https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.Dataset.html
    lgb_valid_dataset = lgb.Dataset(va_x, va_y) #test 데이터를 LGBM용 데이터셋으로 변환합니다.
    model = lgb.train(lgb_params, lgb_train_dataset, valid_sets=[lgb_valid_dataset], verbose_eval=100) #LGBM 모델을 훈련합니다.
    models.append(model) #각 모델을 위에서 정의한 모델 리스트에 입력합니다.
    oof_pred = model.predict(va_x) #validation 데이터를 예측합니다.
    oof_preds[va_idx] = oof_pred #예측한 값을 위에서 만든 DataFrame에 입력합니다.

In [ ]:
np.sqrt(mean_squared_error(df_train.target, oof_preds)) #각 모델로 산출한 validation 데이터의 RMSE 점수를 출력하여 모델의 성능을 확인합니다.

In [ ]:
preds = [] #예측값을 저장할 DataFrame을 만듭니다.
for model in models: #저장했던 모델들을 하나씩 예측하는 for 반복문을 만듭니다.
    pred = model.predict(test_x) #각 모델을 하나씩 예측합니다.
    preds.append(pred) #각 예측값을 위에서 만든 DataFrame에 입력합니다.
preds = np.mean(preds, axis=0) #각 예측값의 평균을 내어 최종 예측값을 완성합니다.

In [ ]:
sub = df_sample.copy() #submission 샘플을 복사합니다.
sub["target"] = preds #샘플의 target column에 최종 예측값을 입력합니다.

sub.to_csv("submission_baseline(simple parameter).csv", index=False) #제출을 위한 csv 파일을 생성합니다.

sub.head() #최종 제출 파일의 앞부분을 확인합니다.

**도움이 되었다면 업보트 upvote 부탁드립니다!!!**